In [1]:
import sys
sys.path.append('/thesis')

In [2]:
import json
import pickle
# import logging
# import time

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from helper.character_encoder import DictionaryCharacterEncoder
from helper.prediction import predict_sequence

from implementation.seq2seq.transformer.utils import masked_loss, masked_accuracy
from implementation.seq2seq.transformer.keras_nlp import prepare_batches
from implementation.seq2seq.transformer.keras_nlp import construct_model_w_teacher_forcing

import tensorflow as tf
import keras_nlp

tf.config.list_physical_devices('GPU')

2023-07-02 15:59:18.452638: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-02 15:59:18.485408: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 15:59:19.975115: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-02 15:59:19.978689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bu

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Input file paths
# jrc_file = '/thesis/data/jrc_person_pairs.csv'
# wikidata_file = '/thesis/data/wikidata_person_to_en_norm.csv'
wikidata_file = '/thesis/data/wikidata_loc_to_en_norm.csv'

model_serialization_path = '/thesis/models/transformer_wikidata_loc_to_en_1'

## Loading & Preprocessing

In [4]:
# Preprocessing
MAX_SEQ_LENGTH = 30  # 40
NUM_SAMPLES = 175000 # 350000
VALIDATION_SPLIT = 0.3
RANDOM_STATE = 1010
BATCH_SIZE = 64

# Model
EMBEDDING_DIM = 64
INTERMEDIATE_DIM = 512
NUM_ENCODER_HEADS = 8
NUM_DECODER_HEADS = 8
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 6

DROPOUT = 0.2

# Training
EPOCHS = 60
CHECKPOINT_FREQ = 10

In [5]:
# pairs_df = pd.read_csv(jrc_file, sep='|', encoding='utf-8')[['input', 'target']]
# pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_SEQ_LENGTH)]
# print('Number of JRC pairs:', len(pairs_df))
# pairs_df2 = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
# pairs_df2 = pairs_df2[(pairs_df2['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df2['target'].str.len() <= MAX_SEQ_LENGTH)]
# print('Number of Wikidata pairs:', len(pairs_df2), '\n')
# pairs_df = pd.concat([pairs_df, pairs_df2]).sample(frac=1, random_state=RANDOM_STATE)

pairs_df = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_SEQ_LENGTH)]
pairs_df

,input,target
0,antarktida,antarctica
1,antiarktis,antarctica
2,antarktika,antarctica
3,antaatika,antarctica
4,antarticu,antarctica
...,...,...
187832,as-sulajjil,as sulayyil
187833,es-suleyil,as sulayyil
187834,olua i refugee settlment,olua i refugee settlement
187835,mont olivine,olivine mountain


In [6]:
dce = DictionaryCharacterEncoder(max_seq_length=MAX_SEQ_LENGTH+2)
train_batches, val_batches = prepare_batches(pairs_df, dce, NUM_SAMPLES, VALIDATION_SPLIT, BATCH_SIZE, RANDOM_STATE)

2023-07-02 15:59:22.742502: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-02 15:59:22.742850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-02 15:59:22.743168: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-02 15:59:23.523081: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-02 15:59:23.523413: I tensorflow/compile

In [7]:
for inputs, targets in train_batches.take(1):
    print(f'Encoder inputs shape: {inputs[0].shape}')
    print(f'Decoder inputs shape: {inputs[1].shape}')
    print(f'Targets shape: {targets.shape}')

Encoder inputs shape: (64, 20)
Decoder inputs shape: (64, 20)
Targets shape: (64, 20)


2023-07-02 15:59:27.150770: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [122500]
	 [[{{node Placeholder/_1}}]]
2023-07-02 15:59:27.151021: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [122500]
	 [[{{node Placeholder/_0}}]]


## Model

In [8]:
transformer = construct_model_w_teacher_forcing(
    num_encoder_layers=NUM_ENCODER_LAYERS, 
    num_decoder_layers=NUM_DECODER_LAYERS, 
    unique_tokens=len(dce.charset),
    max_seq_length=MAX_SEQ_LENGTH,
    embedding_dim=EMBEDDING_DIM,
    intermediate_dim=INTERMEDIATE_DIM,
    encoder_heads=NUM_ENCODER_HEADS,
    decoder_heads=NUM_DECODER_HEADS,
    dropout=DROPOUT
)

In [9]:
transformer.summary()
# transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.compile(
    loss=masked_loss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[masked_accuracy])

Model: "transformer_w_tf"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 64)       2176        ['encoder_inputs[0][0]']         
                                                                                                  
 position_embedding (PositionEm  (None, 32, 64)      2048        ['embedding[0][0]']              
 bedding)                                                                                         
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 32, 64)      0           ['embedding[0][0]'

## Training

In [10]:
steps_per_epoch = len(train_batches)

checkpoint_path = f'{model_serialization_path}/checkpoints/' + 'weights-{epoch:03d}'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                checkpoint_path, 
                monitor='val_masked_accuracy', 
                save_weights_only=True,
                save_freq=int(steps_per_epoch * CHECKPOINT_FREQ), 
                verbose=1)

In [11]:
history = transformer.fit(train_batches, epochs=EPOCHS, validation_data=val_batches, callbacks=[checkpoint])

Epoch 1/60


2023-07-02 15:59:29.108355: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [122500]
	 [[{{node Placeholder/_0}}]]
2023-07-02 15:59:29.108552: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [122500]
	 [[{{node Placeholder/_1}}]]
2023-07-02 15:59:39.717251: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-02 15:59:39.788751: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f5f1772d350 initialized for pl

1915/1915 [==============================] - ETA: 0s - loss: 1.2595 - masked_accuracy: 0.6579

2023-07-02 16:01:26.001961: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [52500]
	 [[{{node Placeholder/_0}}]]
2023-07-02 16:01:26.002268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [52500]
	 [[{{node Placeholder/_1}}]]


1915/1915 [==============================] - 135s 59ms/step - loss: 1.2595 - masked_accuracy: 0.6579 - val_loss: 0.6459 - val_masked_accuracy: 0.8354
Epoch 2/60
1915/1915 [==============================] - 111s 58ms/step - loss: 0.6408 - masked_accuracy: 0.8361 - val_loss: 0.5259 - val_masked_accuracy: 0.8644
Epoch 3/60
1915/1915 [==============================] - 110s 57ms/step - loss: 0.5630 - masked_accuracy: 0.8547 - val_loss: 0.4984 - val_masked_accuracy: 0.8716
Epoch 4/60
1915/1915 [==============================] - 111s 58ms/step - loss: 0.5261 - masked_accuracy: 0.8634 - val_loss: 0.4685 - val_masked_accuracy: 0.8770
Epoch 5/60
1915/1915 [==============================] - 108s 56ms/step - loss: 0.5014 - masked_accuracy: 0.8687 - val_loss: 0.4475 - val_masked_accuracy: 0.8826
Epoch 6/60
1915/1915 [==============================] - 108s 56ms/step - loss: 0.4835 - masked_accuracy: 0.8727 - val_loss: 0.4332 - val_masked_accuracy: 0.8863
Epoch 7/60
1915/1915 [=======================

In [12]:
transformer.save(f'{model_serialization_path}/')

2023-07-02 17:48:15.536717: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,64]
	 [[{{node inputs}}]]
2023-07-02 17:48:15.546673: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,64]
	 [[{{node decoder_sequence}}]]
2023-07-02 17:48:15.557327: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,64]
	 [[{

INFO:tensorflow:Assets written to: /thesis/models/transformer_wikidata_loc_to_en_1/assets


INFO:tensorflow:Assets written to: /thesis/models/transformer_wikidata_loc_to_en_1/assets


In [13]:
with open(model_serialization_path + '/train_history.p', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

train_config = {
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'num_samples': NUM_SAMPLES,
    'max_seq_length': MAX_SEQ_LENGTH,
    'random_state': RANDOM_STATE,
    'validation_split': VALIDATION_SPLIT,
    'encoder_layers': NUM_ENCODER_LAYERS,
    'decoder_layers': NUM_DECODER_LAYERS,
    'encoder_heads': NUM_ENCODER_HEADS,
    'decoder_heads': NUM_DECODER_HEADS,
    'dropout': DROPOUT,
    'embedding_dim': EMBEDDING_DIM,
    'intermediate_dim': INTERMEDIATE_DIM
}

with open(model_serialization_path + '/config.p', 'wb') as file_pi:
    pickle.dump(train_config, file_pi)

In [14]:
transformer = tf.keras.models.load_model(f'{model_serialization_path}/', custom_objects={'masked_loss': masked_loss, 'masked_accuracy': masked_accuracy})

## Prediction

In [15]:
def decode_sequences(input_sentence):
     # Tokenize the encoder input.
    encoder_input_tokens = tf.keras.utils.pad_sequences(
        dce.to_ids([input_sentence], insert_markers=True), 
        padding='post', 
        maxlen=MAX_SEQ_LENGTH+2
    )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def token_probability_fn(decoder_input_tokens):
        return transformer([encoder_input_tokens, decoder_input_tokens])[:, -1, :]


    prompt = tf.fill((1, 1), dce.char_index['\t'])
    generated_tokens = keras_nlp.utils.top_p_search(
        token_probability_fn,
        prompt,
        p=0.1,
        max_length=MAX_SEQ_LENGTH,
        end_token_id=dce.char_index['\n'],
    )
    generated_sentences = ''.join([dce.inverse_char_index[tkn] for tkn in generated_tokens.numpy()[0]])
    return generated_sentences.strip()

names = [
    'samuel meyer',
    'dmitry medvedev',
    'paulo ricardo',
    'zouheir al qaissi',
    'tarek al bichri',
    'thorsten brotzmann'
]

for s in names:
    translated = decode_sequences(s)
    print(s)
    print(translated)
    print()


samuel meyer
samuel meyer

dmitry medvedev
dmitry medvedev

paulo ricardo
ricardo paul

zouheir al qaissi
zouheir al-qaisi

tarek al bichri
tarek al-bikhri

thorsten brotzmann
thorsten brotzmann

